[![Open In Wolfram Cloud](https://raw.githubusercontent.com/gvarnavi/generative-art-iap/master/PR/wolfram-badge.svg)](https://www.wolframcloud.com/obj/gvarnavi/Published/01X_mathematical-fractals.nb)

# Mathematical Fractals

"A fractal is a way of seeing infinity." ~ *Benoit Mandelbrot*

## Julia Set

We investigate a simple quadratic mapping given by:

$$ z_{n+1} = z_n^2 + c $$

We're interested in the set of "exceptional points", z.  
I.e. those which do not escape to complex infinity after a quadratic mapping is applied repeatedly.

**Note:** This is the simplest case of an 'orbit trap' - we'll investigate these further later today.

In [ ]:
naiveJulia[{zReal_, zIm_}, c_, maxIters_ : 100, escapeRadius_ : 1] :=
 
 NestWhile[#^2 + c &, zReal + zIm I, 
  Abs[#] < escapeRadius &, 1, 
  maxIters]

In [ ]:
ArrayPlot[
 Table[naiveJulia[{r, i}, I, 10, 2],
 {i, -1.5, 1.5, 0.005},
 {r, -1.5, 1.5, 0.005}
 ],ImageSize->Small,DataReversed->True]

Compare with built-in Julia-set function:

In [ ]:
JuliaSetPlot[I,ImageSize->Small]

This particular Julia set is called 'the dendritic fractal' and is related to the second fractal we'll investigate!

## Mandelbrot Set
The connected set of these "exceptional points" in the Julia Set forms the Mandelbrot set.  
This is again given by the same quadratic mapping:

$$ z_{n+1} = z_n^2 + c $$

The difference being we'll now iterate over the complex number c with an initial starting point, namely 0. + 0 I.

In [ ]:
naiveMandelBrot[{cReal_, cIm_}, maxIters_ : 100, escapeRadius_ : 2] :=
  If[Length[
    NestWhileList[#^2 + (cReal + cIm I) &,
     0. + 0. I, 
     Abs[#] < escapeRadius &, 1, maxIters]] > maxIters, 1, 0]

In [ ]:
ArrayPlot[
 Table[naiveMandelBrot[{r, i}, 10, 2], 
 {i, -2, 2, 0.004},
 {r, -2, 2, 0.004}],ImageSize->Small,DataReversed->True]

We'll add a bit more detail by counting the iterations on the boundary (scaled by the max iteration), increasing the number of iterations and adding color.

In [ ]:
lessNaiveMandelBrot[{cReal_, cIm_}, maxIters_ : 100, escapeRadius_ : 2] := 
 Sqrt[Length[
    NestWhileList[#^2 + (cReal + cIm I) &, 
    0. + 0. I, 
    Abs[#] < escapeRadius &, 1, maxIters]]/maxIters]

In [ ]:
ArrayPlot[
 Table[lessNaiveMandelBrot[{r, i}, 100, 2], 
 {i, -2, 2, 0.004},
 {r, -2, 2, 0.004}], 
 ColorFunction -> ColorData["SunsetColors"],
 DataReversed->True,
 ImageSize->Small]

Compare this with the built-in Mandelbrot-set function:

In [ ]:
MandelbrotSetPlot[ImageSize->Small]

The 'naive' implementation is getting rather slow.  
Let's compile it in C!

In [ ]:
mnd = Compile[{{maxiter, _Integer}, {zinit, _Complex}, {dt, _Real}}, 
  Module[{z, c, iters},
   Table[
    
    z = zinit;
    c = cr + I*ci;
    iters = 0.;
    
    While[(iters < maxiter) && (Abs@z < 2), 
     
     iters++;
     z = z^2 + c];
     
    Sqrt[iters/maxiter],
    {ci, -2, 2, dt},
    {cr, -2, 2, dt}]],
    
  CompilationTarget -> "C", 
  RuntimeOptions -> "Speed"]

In [ ]:
Timing[lst = mnd[100, 0 + 0 I, .01];]
ArrayPlot[lst, 
ColorFunction -> ColorData["SunsetColors"],
ImageSize->Small]

We get  can get interesting behaviour by changing the starting point dynamically.

In [ ]:
Timing[lst = mnd[100, .5 + .5 I, .01];]
ArrayPlot[lst,
ColorFunction -> ColorData["SunsetColors"],
ImageSize->Small]

**Alert:** Unfortunately the Jupyter front-end for Mathematica has limited support for `Dynamic` functionality. We'll make a movie here, see the Wolfram cloud notebook for an interactive version

In [ ]:
frames=Table[
 ArrayPlot[mnd[100, (1. + 1. I) p, 0.01],
  ColorFunction -> ColorData["SunsetColors"], Frame -> False],
 {p, Subdivide[30]}];

Export["gifs/mandelbrot-blowing-in-the-wind-animation.gif", frames,
ImageSize->300,"AnimationRepetitions" -> Infinity]

![mandelbrot](https://raw.githubusercontent.com/gvarnavi/generative-art-iap/master/01.26-Wednesday/gifs/mandelbrot-blowing-in-the-wind-animation.gif "mandelbrot")

## Complex Polynomial Roots
In mathematics, a Littlewood polynomial is a polynomial all of whose coefficients are +1 or -1.  
The complex roots of such polynomials often show fractal behavior!

In [ ]:
Graphics[{PointSize[Tiny], 
  Point@Flatten[(({Re[z], Im[z]} /. #) & /@ 
        NSolve[z^Range[0, 11] . # == 0, z]) 
        & /@ Tuples[{-1, 1}, 12], 
    1]}, ImageSize -> 400]

This is a very "terse" one-liner. Let's analyze it:

In [ ]:
z^Range[0, 5]
z^Range[0, 5] . {3, 4, 0, 1, 2, 6}

The above two lines are a compact line of generating a complex polynomial.  
To make Littlewood polynomials in particular, say of degree 11, we take all length-12 combinations of -1 and 1:

In [ ]:
Length[tups=Tuples[{-1, 1}, 12]]
RandomChoice[tups]

Putting it all together:

In [ ]:
z^Range[0, 11] . RandomChoice[Tuples[{-1, 1}, 12]]

We then proceed to solve for the complex roots of the Littlewood polynomial:

In [ ]:
NSolve[
z^Range[0, 11] . RandomChoice[Tuples[{-1, 1}, 12]] == 0,
z]